In [1]:
## Data scrapping prpm
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
PRPMscrap = PRPMScraper()

## find sinonim still buggy (will work if got time)
result = PRPMscrap.findWordMetaData('selain')
print(result)

[<b>Bersinonim dengan </b>]
{'selain': {'meanings': ['1. hanya, kecuali: ia tidak dapat berbuat apa-apa ~ membiarkan sahaja; 2. di samping, bersama-sama dgn: ~ ceramah, diadakan juga pembacaan sajak;', ', = lain drpd 1 di samping: ~ berlatih kawad, mereka belajar undang-undang juga. 2 kecuali; hanya: Emaknya tidak dpt berbuat apa-apa, ~ menangis.'], 'sinonim': ['berbeza']}}


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import os
os.environ['DISPLAY'] = ':0'

class BHarianScraper:
    def __init__(self,category = None):
        self.category = category
        self.options = Options()
        self.options.add_argument("--headless")  # Run in headless mode
        self.options.add_argument("--disable-gpu")  # Disable GPU acceleration
        self.options.add_argument("--no-sandbox")  # Bypass OS security model
        self.options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
        self.options.add_argument("--disable-extensions")  # Disable extensions
        self.options.add_argument("--disable-software-rasterizer")  # Use software rendering
        self.options.add_argument("--incognito") 
        self.options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        self.service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.service, options=self.options)

    
    def getScrapResult(self, category = None):
        #if category not specified raise exceptions
        if category == None and self.category == None:
            raise CategoryError(category,f"Category is invalid. Category=\'{self.category}\'. Please specify category in the form of url appending string.")

        #Assign new cat if new
        self.category = category if category is not None else self.category

        #Web Crawl Begins
        try:
            self.driver.get(f"https://www.bharian.com.my/{self.category}")
            self.driver.implicitly_wait(10)

            # try:
            #     # Wait for the div with class 'article-teaser' to appear
            #     WebDriverWait(self.driver, 10).until(
            #         EC.presence_of_element_located((By.CLASS_NAME, 'article-teaser'))
            #     )
            #     print("Article teaser found.")
            # except Exception as e:
            #     print(f"Error: {e}")
            
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            print(soup)
            articles = soup.find_all(
                lambda tag: tag.name == 'div' 
                and 'article-teaser' in tag.get('class', []) 
                and 'd-block' not in tag.get('class', [])
            )
            for content in articles:
                print(content.prettify())
            print(len(articles))
        finally:
            self.driver.quit()

        
            

class CategoryError(Exception):
    def __init__(self, category, message="Category is invalid."):
        self.category = category
        self.message = message
        super().__init__(self.message)     

bharianScraper = BHarianScraper()
bharianScraper.getScrapResult(category='sukan/bola')

<html lang="en"><head>
<link crossorigin="" href="https://ad.crwdcntrl.net" rel="dns-prefetch"/>
<link crossorigin="" href="https//www.googletagservices.com" rel="dns-prefetch"/>
<link crossorigin="" href="https//www.googletagmanager.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://fonts.googleapis.com" rel="dns-prefetch"/>
<link href="https://tags.crwdcntrl.net" rel="preconnect"/>
<link href="https://bcp.crwdcntrl.net" rel="preconnect"/>
<link href="https://tags.crwdcntrl.net" rel="dns-prefetch"/>
<link href="https://bcp.crwdcntrl.net" rel="dns-prefetch"/>
<link as="style" href="https://assets.bharian.com.my/assets/css/libraries.css?id=7979572225756f8aee4c" rel="preload"/>
<link as="style" href="https://assets.bharian.com.my/assets/css/desktop/app.css?id=cba796c03c8280aaf711" rel="preload"/>
<link as="script" href="https://assets.bharian.com.my/assets/js/desktop/app.js?id=97df5a01caeb311557fd" rel="preload"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initi